In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [2]:
class network_structure():
    def __init__(self, edges_nodes_matrix):
        # Structure of edges_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an edge.
        
        assert edges_nodes_matrix.shape[1] == 2, "We must have edges_nodes_matrix.shape[1] == 2"
        
        self.edges_nodes_matrix = edges_nodes_matrix
        self.nodes = np.unique(self.edges_nodes_matrix.flatten())
        self.edges = np.array(range(self.edges_nodes_matrix.shape[0]))
        self.num_nodes = len(self.nodes)
        self.num_edges = len(self.edges)
        
        self.construct_nodes_dict()
        self.construct_edges_dict()
#         self.construct_depth_dicts()
#         self.construct_height_dicts()
    
        origin_nodes = [node for node in self.nodes if self.nodes_dict[node]['incoming_edges'] == []]
        destination_nodes = [node for node in self.nodes if self.nodes_dict[node]['outgoing_edges'] == []]
        
        assert len(origin_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        assert len(destination_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        self.origin_node = origin_nodes[0]
        self.destination_node = destination_nodes[0]
        self.intermediate_nodes = [node for node in self.nodes if node not in [self.origin_node, self.destination_node]]
        
        self.start_edges = self.nodes_dict[self.origin_node]["outgoing_edges"]
        self.end_edges = self.nodes_dict[self.destination_node]["incoming_edges"]
        
        self.construct_routes_dict()
    
        return

    
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_edges'] = list((self.edges_nodes_matrix[:, 1] == node).nonzero()[0])
            self.nodes_dict[node]['outgoing_edges'] = list((self.edges_nodes_matrix[:, 0] == node).nonzero()[0])
            
            self.nodes_dict[node]['previous_nodes'] = list(set(self.edges_nodes_matrix[edge, 0] for edge in self.nodes_dict[node]['incoming_edges']))
            self.nodes_dict[node]['next_nodes'] = list(set(self.edges_nodes_matrix[edge, 1] for edge in self.nodes_dict[node]['outgoing_edges']))
        
        return

    def construct_edges_dict(self):
        self.edges_dict = {}

        for edge in self.edges:
            self.edges_dict[edge] = {}

        for node in self.nodes:
            if self.nodes_dict[node]['incoming_edges'] != []:
                for edge in self.nodes_dict[node]['incoming_edges']:
                    self.edges_dict[edge]['end_node'] = node

            if self.nodes_dict[node]['outgoing_edges'] != []:
                for edge in self.nodes_dict[node]['outgoing_edges']:
                    self.edges_dict[edge]['start_node'] = node

        for edge in self.edges:
            self.edges_dict[edge] = {}
            start_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['outgoing_edges']]))
            end_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['incoming_edges']]))
            self.edges_dict[edge]['start_node'] = start_node
            self.edges_dict[edge]['end_node'] = end_node
            self.edges_dict[edge]['incoming_edges'] = self.nodes_dict[start_node]['incoming_edges']
            self.edges_dict[edge]['outgoing_edges'] = self.nodes_dict[end_node]['outgoing_edges']
        return
    
    def construct_routes_dict(self):
        self.routes = [[edge] for edge in self.start_edges]
        
        route_construction_finished = False
        while route_construction_finished is False:
            route_construction_finished = True
            routes_temp = []
            for route_segment in self.routes:
                
#                 print()
#                 print("route_segment:", route_segment)
#                 print("route_segment[-1]:", route_segment[-1])
#                 print("self.edges_dict[route_segment[-1]]:", self.edges_dict[route_segment[-1]])
#                 print("self.edges_dict[route_segment[-1]][outgoing_edges]:", self.edges_dict[route_segment[-1]]["outgoing_edges"])
# #                 print()
                
                if self.edges_dict[route_segment[-1]]["outgoing_edges"] != []:
                    route_construction_finished = False
                    for edge in self.edges_dict[route_segment[-1]]["outgoing_edges"]:
                        routes_temp.append(route_segment + [edge])
                else:
                    routes_temp.append(route_segment)
                
#                 print("routes_temp:", routes_temp)
                
                
            if route_construction_finished is True:
                break
            self.routes = routes_temp
            
#             print()
#             print("self.routes:", self.routes)
#             print()

        return
    
    def print_nodes_dict(self):
        print()
        print("Printing nodes:")
        for node_key, node_value in self.nodes_dict.items():
            print()
            print("Node", node_key)
            for key, value in node_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_edges_dict(self):
        print()
        print("Printing edges:")
        for edge_key, edge_value in self.edges_dict.items():
            print()
            print("Edge", edge_key)
            for key, value in edge_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_routes(self):
        print()
        print("Printing routes:")
        print()
        for route_index, route_edges in enumerate(self.routes):
            print("Route", route_index, ":", route_edges)
        return
    
    def print_all(self):
        print()
        print("Printing network information:")
        print()
        print("Origin node:", self.origin_node)
        print("Destination node:", self.destination_node)
        print("Number of nodes:", self.num_nodes)
        print()
        print("Starting edges:", self.start_edges)
        print("Ending edges:", self.end_edges)
        print("Number of edges:", self.num_edges)
        
        self.print_nodes_dict()
        self.print_edges_dict()
        self.print_routes()
        
        return
    

In [3]:
# list_1 = [1, 2]
# list_2 = [3, 4, 5]
# list_1 = list_1 + [6]
# list_2.append(list_1)
# list_2

In [4]:
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
netw_example = network_structure(edges_nodes_matrix)

# print("netw_example.edges:", netw_example.edges)
# print()
# print("netw_example.nodes:", netw_example.nodes)
# print()
# print("netw_example.edges_dict:", netw_example.edges_dict)
# print()
# print("netw_example.nodes_dict:", netw_example.nodes_dict)
# print()
# print("netw_example.routes:", netw_example.routes)
# print()

# netw_example.print_nodes_dict()
# netw_example.print_edges_dict()
# netw_example.print_routes()

netw_example.print_all()
# netw_example.intermediate_nodes



Printing network information:

Origin node: 0
Destination node: 3
Number of nodes: 4

Starting edges: [0, 1]
Ending edges: [3, 4]
Number of edges: 5

Printing nodes:

Node 0
incoming_edges : []
outgoing_edges : [0, 1]
previous_nodes : []
next_nodes : [1, 2]

Node 1
incoming_edges : [0]
outgoing_edges : [2, 3]
previous_nodes : [0]
next_nodes : [2, 3]

Node 2
incoming_edges : [1, 2]
outgoing_edges : [4]
previous_nodes : [0, 1]
next_nodes : [3]

Node 3
incoming_edges : [3, 4]
outgoing_edges : []
previous_nodes : [1, 2]
next_nodes : []


Printing edges:

Edge 0
start_node : 0
end_node : 1
incoming_edges : []
outgoing_edges : [2, 3]

Edge 1
start_node : 0
end_node : 2
incoming_edges : []
outgoing_edges : [4]

Edge 2
start_node : 1
end_node : 2
incoming_edges : [0]
outgoing_edges : [4]

Edge 3
start_node : 1
end_node : 3
incoming_edges : [0]
outgoing_edges : []

Edge 4
start_node : 2
end_node : 3
incoming_edges : [1, 2]
outgoing_edges : []


Printing routes:

Route 0 : [0, 2, 4]
Route 1 : [

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [5]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [9]:
def welfare_obj(network, lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_ex[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
        
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p, e] * (x_ex[e] ** p) for p in range(5))
        ell_gp[e] = sum(a_gp[p, e] * (x_gp[e] ** p) for p in range(5))
    
    for e in range(num_edges):
        obj_E = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el)) \
                    + sum(y_el[(g, e, k)] * v_E_array[g] * ell_ex[e] for g in range(num_el) for k in [0, 1]) \
                    + sum(y_el[(g, e, 2)] * v_E_array[g] * ell_gp[e] for g in range(num_el))
        obj_I = sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in)) \
                    + sum(y_in[(g, e, 0)] * v_I_array[g] * ell_ex[e] for g in range(num_in)) \
                    + sum(y_in[(g, e, 1)] * v_I_array[g] * ell_gp[e] for g in range(num_in))
        obj_R = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el)) \
                    + sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare

def latency_total(network, tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_ex[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
    
    # Add up latency values:
    latency_val = 0.0
    
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p, e] * (x_ex[e] ** p) for p in range(5))
        ell_gp[e] = sum(a_gp[p, e] * (x_gp[e] ** p) for p in range(5))
        
        latency_val += x_ex[e] * ell_ex[e] + x_gp[e] * ell_gp[e]
    
    return latency_val



In [10]:
def proj_tau_B_11(network, tau, B, tau_max = 1.0, B_max = 1.0):
    
    num_edges = len(network.edges)
    num_nodes = len(network.nodes)
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()
    
    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    tau_feas = cp.Variable(num_edges)
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)
        
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones(num_edges)]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= sum(tau_feas[edge] for edge in route) for route in network.routes]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

In [11]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [12]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(network, tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el[(g, e, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in[(g, e, k)] = cp.Variable(1)

    x_ex = cp.Variable(network.num_edges)
    x_gp = cp.Variable(network.num_edges)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        func += 1/5 * a_ex[4, e] * cp.power(x_ex[e], 5)
        func += 1/4 * a_ex[3, e] * cp.power(x_ex[e], 4)
        func += 1/3 * a_ex[2, e] * cp.power(x_ex[e], 3)
        func += 1/2 * a_ex[1, e] * cp.power(x_ex[e], 2)
        func += a_ex[0, e] * x_ex[e]
        func += 1/5 * a_gp[4, e] * cp.power(x_gp[e], 5)
        func += 1/4 * a_gp[3, e] * cp.power(x_gp[e], 4)
        func += 1/3 * a_gp[2, e] * cp.power(x_gp[e], 3)
        func += 1/2 * a_gp[1, e] * cp.power(x_gp[e], 2)
        func += a_gp[0, e] * x_gp[e]
        
        func += sum(tau[e] * y_el[(g, e, 1)] / v_E_array[g] for g in range(num_el))
        func += sum(tau[e] * y_in[(g, e, 0)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k)] >= 0.0 for g in range(num_el) for e in network.edges for k in [0, 1, 2]]
    constraints += [y_in[(g, e, k)] >= 0.0 for g in range(num_in) for e in network.edges for k in [0, 1]]
        
    constraints += [sum(y_el[(g, e, k)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el)]
    constraints += [sum(y_in[(g, e, k)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el)]
        constraints += [sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in)]
    
    for e in network.edges:
        constraints += [x_ex[e] == sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1]) \
                                   + sum(y_in[(g, e, 0)] for g in range(num_in)) ]
        constraints += [x_gp[e] == sum(y_el[(g, e, 2)] for g in range(num_el)) \
                                   + sum(y_in[(g, e, 1)] for g in range(num_in)) ]
    
    constraints += [sum(y_el[(g, e, 0)]  * tau[e] for e in network.edges) <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el_values[(g, e, k)] = max(np.round(y_el[(g, e, k)].value[0], decimals=6) , 0.0)

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in_values[(g, e, k)] = max(np.round(y_in[(g, e, k)].value[0], decimals=6), 0.0)

    return y_el_values, y_in_values


In [13]:
time_1 = time.time()

edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
network_Braess = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess

## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional n ,\
## (e.g., 3d or 4d) arrays:\n,

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 10.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
flow_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
tau_max = sum(a[k] * (flow_max ** k) for k in range(5))
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros((network.num_edges, num_iters))
tau_perturbed = np.zeros((network.num_edges, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, 0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges +1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, i] = tau[:, i] + delta[i] * w_i[:-1]
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, i] < B_perturbed[i] or tau_perturbed[:, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, i], B_perturbed[i] = proj_tau_B_11(network, tau_perturbed[:, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[:, i]:", tau[:, i])
    print("B[i]:", B[i])
    print("tau_perturbed[:, i]:", tau_perturbed[:, i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(network, tau = tau[:, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(network, tau = tau_perturbed[:, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau[:, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare):\n", tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare))
    
    tau[:, i+1] = tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
    print("welfare:", welfare)
    print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, i+1], B[i+1] = proj_tau_B_11(network, tau[:, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, i-diffs_num_cols : i-1] - tau[:, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, 0:10]:", tau[:, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [ 0.30069725 -0.43969271  0.02140243 -0.81579784 -0.19962651  0.1020616 ]
tau[:, i]: [907.2 907.2 907.2 907.2 907.2]
B[i]: 388.8
tau_perturbed[:, i]: [907.8379 906.2673 907.2454 905.4694 906.7765]
B_perturbed[i]: 389.0165

Iter: 1
w_i: [-0.43455124 -0.50522599 -0.44914282 -0.53986688 -0.10092887 -0.22921554]
tau[:, i]: [905.5395 909.6281 907.0818 911.7051 908.3024]
B[i]: 388.2364
tau_perturbed[:, i]: [904.7643 908.7269 906.2806 910.7421 908.1224]
B_perturbed[i]: 387.8275

Iter: 2
w_i: [ 0.19260145  0.77537182  0.1640677   0.53242966 -0.07320034 -0.21434873]
tau[:, i]: [893.4201 895.5376 894.5555 896.6485 905.4876]
B[i]: 381.8437
tau_perturbed[:, i]: [893.7305 896.7874 894.82   897.5067 905.3696]
B_perturbed[i]: 381.4982

Iter: 3
w_i: [ 0.20138492 -0.55482207  0.01871804 -0.26541319 -0.70092345  0.29921293]
tau[:, i]: [893.1753 894.5522 894.347  895.9718 905.5806]
B[i]: 382.1161
tau_perturbed[:, i]: [893.4774 893.72   894.3751 895.5737 904.5292]
B_perturbed[i]: 382.5649

I

tau_diffs: [1.13221444 1.10198266 0.57370887 1.23813129]
B_diffs: [0.3087 0.6362 0.0867 0.8264]

Iter: 26
w_i: [ 0.28641406 -0.29505356 -0.66704479 -0.25866332 -0.45269647 -0.33781774]
tau[:, i]: [910.4637 898.7773 926.0498 909.3568 849.6206]
B[i]: 334.2807
tau_perturbed[:, i]: [910.7302 898.5027 925.429  909.1161 849.1993]
B_perturbed[i]: 333.9663
tau_diffs: [1.10198266 0.57370887 1.23813129 0.1731519 ]
B_diffs: [ 0.6362  0.0867  0.8264 -0.0205]

Iter: 27
w_i: [-0.34617074 -0.07572983  0.25749504 -0.27791304 -0.24580308 -0.81882373]
tau[:, i]: [910.5821 898.6554 925.7741 909.2499 849.4335]
B[i]: 334.1411
tau_perturbed[:, i]: [910.2629 898.5856 926.0116 908.9936 849.2068]
B_perturbed[i]: 333.386
tau_diffs: [0.57370887 1.23813129 0.1731519  1.21453045]
B_diffs: [ 0.0867  0.8264 -0.0205  0.3792]

Iter: 28
w_i: [-0.38443978 -0.4170341  -0.01084831 -0.3330043   0.03849731 -0.7521948 ]
tau[:, i]: [909.9703 898.5215 926.2292 908.7587 848.999 ]
B[i]: 332.6938
tau_perturbed[:, i]: [909.6189 89

tau_diffs: [0.34962653 0.15248052 0.84279114 0.38952363]
B_diffs: [0.0045 0.036  1.1842 0.0844]

Iter: 50
w_i: [-0.17562267 -0.35971951  0.35775732 -0.09991808 -0.32151648 -0.7735708 ]
tau[:, i]: [905.6899 896.4605 924.2917 907.748  852.8857]
B[i]: 317.8227
tau_perturbed[:, i]: [905.5505 896.175  924.5757 907.6687 852.6305]
B_perturbed[i]: 317.2086
tau_diffs: [0.15248052 0.84279114 0.38952363 0.53229474]
B_diffs: [0.036  1.1842 0.0844 0.2863]

Iter: 51
w_i: [-0.52725739 -0.4176238  -0.38419918  0.33566398  0.08117486  0.52983142]
tau[:, i]: [905.498  896.0674 924.6826 907.6388 852.5344]
B[i]: 316.9774
tau_perturbed[:, i]: [905.0815 895.7375 924.3791 907.904  852.5985]
B_perturbed[i]: 317.3959
tau_diffs: [0.84279114 0.38952363 0.53229474 0.73713094]
B_diffs: [1.1842 0.0844 0.2863 0.7728]

Iter: 52
w_i: [ 0.4282983  -0.21400196 -0.35483141 -0.2873799   0.19975674 -0.7227506 ]
tau[:, i]: [905.9981 896.4635 925.047  907.3204 852.4574]
B[i]: 316.4748
tau_perturbed[:, i]: [906.3348 896.2953 

tau_diffs: [0.55032205 0.65141558 0.42761417 0.65101473]
B_diffs: [0.3467 0.3527 0.1768 0.2795]

Iter: 74
w_i: [ 0.36397946  0.13448018  0.19872281 -0.07648254  0.47638812 -0.75970261]
tau[:, i]: [904.7897 897.5252 923.2265 906.7861 854.7486]
B[i]: 309.4936
tau_perturbed[:, i]: [905.0521 897.6221 923.3697 906.731  855.092 ]
B_perturbed[i]: 308.946
tau_diffs: [0.65141558 0.42761417 0.65101473 0.1192351 ]
B_diffs: [0.3527 0.1768 0.2795 0.0089]

Iter: 75
w_i: [0.87240225 0.42597356 0.12810891 0.19037751 0.01675646 0.0672649 ]
tau[:, i]: [905.2373 897.6906 923.4709 906.692  855.3344]
B[i]: 308.5594
tau_perturbed[:, i]: [905.8641 897.9966 923.5629 906.8288 855.3464]
B_perturbed[i]: 308.6077
tau_diffs: [0.42761417 0.65101473 0.1192351  0.54007823]
B_diffs: [0.1768 0.2795 0.0089 0.1094]

Iter: 76
w_i: [ 0.50245817  0.36963372  0.4103384   0.27874192  0.59727572 -0.08996524]
tau[:, i]: [905.2314 897.6877 923.47   906.6907 855.3343]
B[i]: 308.5589
tau_perturbed[:, i]: [905.5912 897.9524 923.763

tau_diffs: [0.27217522 0.19338775 0.22711539 0.55577981]
B_diffs: [0.0733 0.0326 0.0068 0.3878]

Iter: 98
w_i: [-0.32876278 -0.61202835 -0.50884501 -0.09111432  0.21643816 -0.45085007]
tau[:, i]: [905.6913 897.4329 923.1316 907.1431 855.7253]
B[i]: 305.5761
tau_perturbed[:, i]: [905.4702 897.0213 922.7894 907.0818 855.8709]
B_perturbed[i]: 305.2729
tau_diffs: [0.19338775 0.22711539 0.55577981 0.08049043]
B_diffs: [ 0.0326  0.0068  0.3878 -0.0097]

Iter: 99
w_i: [-0.5640518   0.57176375  0.01233498 -0.50075966 -0.04081681  0.31992715]
tau[:, i]: [905.5311 897.1347 922.8837 907.0987 855.8308]
B[i]: 305.3564
tau_perturbed[:, i]: [905.1527 897.5183 922.892  906.7628 855.8034]
B_perturbed[i]: 305.571
tau_diffs: [0.22711539 0.55577981 0.08049043 0.10436182]
B_diffs: [ 0.0068  0.3878 -0.0097  0.0025]

Iter: 100
w_i: [ 0.0021215   0.06124879  0.19442808 -0.73187953  0.60378565 -0.24132353]
tau[:, i]: [905.7008 896.9626 922.88   907.2494 855.8431]
B[i]: 305.2601
tau_perturbed[:, i]: [905.7022 8

...

tau_diffs: [0.06930188 0.01535643 0.03900564 0.18955005]
B_diffs: [ 0.009  -0.0004  0.0045  0.1433]

Iter: 889
w_i: [-0.30488424  0.49464502  0.20308587  0.44608076 -0.5252818   0.38238595]
tau[:, i]: [893.0923 907.7003 928.6264 902.0696 864.9279]
B[i]: 241.1829
tau_perturbed[:, i]: [892.9739 907.8924 928.7053 902.2428 864.7239]
B_perturbed[i]: 241.3314
tau_diffs: [0.01535643 0.03900564 0.18955005 0.10011483]
B_diffs: [-0.0004  0.0045  0.1433  0.0167]

Iter: 890
w_i: [-0.17171748 -0.5917241   0.06593688 -0.6938941   0.33359583  0.15248878]
tau[:, i]: [893.1459 907.6133 928.5907 901.9911 865.0203]
B[i]: 241.1156
tau_perturbed[:, i]: [893.0792 907.3835 928.6163 901.7217 865.1498]
B_perturbed[i]: 241.1748
tau_diffs: [0.03900564 0.18955005 0.10011483 0.08433629]
B_diffs: [0.0045 0.1433 0.0167 0.0178]

Iter: 891
w_i: [-0.40789357  0.19923932 -0.43746658  0.74327841 -0.20213836 -0.09605625]
tau[:, i]: [893.1529 907.6373 928.588  902.0192 865.0068]
B[i]: 241.1094
tau_perturbed[:, i]: [892.994

tau_diffs: [0.02522856 0.09538192 0.10370723 0.16107622]
B_diffs: [0.0021 0.0174 0.0152 0.0646]

Iter: 913
w_i: [-0.16268497 -0.31026169  0.54557464 -0.45249224  0.10156092  0.60378452]
tau[:, i]: [893.359  907.5568 928.5225 901.8543 865.56  ]
B[i]: 240.0897
tau_perturbed[:, i]: [893.2962 907.4371 928.733  901.6797 865.5992]
B_perturbed[i]: 240.3226
tau_diffs: [0.09538192 0.10370723 0.16107622 0.14928376]
B_diffs: [0.0174 0.0152 0.0646 0.1026]

Iter: 914
w_i: [ 0.37429961  0.12172149 -0.05542605  0.23138435 -0.5275735  -0.71424023]
tau[:, i]: [893.3927 907.6211 928.4094 901.9481 865.5389]
B[i]: 239.9645
tau_perturbed[:, i]: [893.5371 907.668  928.388  902.0373 865.3354]
B_perturbed[i]: 239.689
tau_diffs: [0.10370723 0.16107622 0.14928376 0.05781877]
B_diffs: [0.0152 0.0646 0.1026 0.0098]

Iter: 915
w_i: [-0.32846018 -0.04884645  0.32375048 -0.50238523 -0.72170246 -0.10801943]
tau[:, i]: [893.4792 907.6492 928.3966 902.0016 865.417 ]
B[i]: 239.7994
tau_perturbed[:, i]: [893.3525 907.630

tau_diffs: [0.16405054 0.13524182 0.13554254 0.07556388]
B_diffs: [0.1332 0.0903 0.089  0.0038]

Iter: 936
w_i: [ 0.04937441  0.48176307 -0.15721963 -0.67471473 -0.47758025 -0.23963645]
tau[:, i]: [893.4013 907.808  928.4291 901.8883 865.283 ]
B[i]: 238.5303
tau_perturbed[:, i]: [893.4202 907.9927 928.3688 901.6296 865.0999]
B_perturbed[i]: 238.4384
tau_diffs: [0.13524182 0.13554254 0.07556388 0.00463465]
B_diffs: [0.0903 0.089  0.0038 0.0001]

Iter: 937
w_i: [ 0.52584186 -0.6840631   0.27814993 -0.06826819 -0.27405319  0.31371155]
tau[:, i]: [893.4041 907.8356 928.4201 901.8497 865.2557]
B[i]: 238.5166
tau_perturbed[:, i]: [893.6057 907.5734 928.5267 901.8235 865.1507]
B_perturbed[i]: 238.6369
tau_diffs: [0.13554254 0.07556388 0.00463465 0.15386764]
B_diffs: [8.900e-02 3.800e-03 1.000e-04 2.001e-01]

Iter: 938
w_i: [-0.72207043 -0.10675396  0.6458985  -0.07897044  0.00401465  0.20923807]
tau[:, i]: [893.3508 907.9049 928.3919 901.8566 865.2835]
B[i]: 238.4848
tau_perturbed[:, i]: [893

tau_diffs: [0.01121918 0.17092893 0.01750971 0.05892606]
B_diffs: [ 0.0006  0.1106 -0.0014  0.0137]

Iter: 960
w_i: [-0.20729798  0.24545237 -0.48386792 -0.72261624  0.33960817  0.15857039]
tau[:, i]: [893.587  908.0557 928.428  901.8078 865.5499]
B[i]: 237.7545
tau_perturbed[:, i]: [893.508  908.1492 928.2436 901.5325 865.6793]
B_perturbed[i]: 237.8149
tau_diffs: [0.17092893 0.01750971 0.05892606 0.14701469]
B_diffs: [ 0.1106 -0.0014  0.0137  0.153 ]

Iter: 961
w_i: [ 0.05675214 -0.51422205 -0.31972072  0.59050723 -0.50229756  0.17068072]
tau[:, i]: [893.5957 908.0455 928.4482 901.838  865.5357]
B[i]: 237.7479
tau_perturbed[:, i]: [893.6173 907.8496 928.3264 902.0629 865.3444]
B_perturbed[i]: 237.8129
tau_diffs: [0.01750971 0.05892606 0.14701469 0.01079861]
B_diffs: [-0.0014  0.0137  0.153   0.0007]

Iter: 962
w_i: [ 0.06564162 -0.00996034  0.16958816 -0.51938308  0.05913202  0.83281237]
tau[:, i]: [893.591  908.088  928.4746 901.7892 865.5772]
B[i]: 237.7338
tau_perturbed[:, i]: [893

tau_diffs: [0.09657909 0.00917497 0.07099655 0.03663086]
B_diffs: [0.0516 0.0009 0.0207 0.0018]

Iter: 984
w_i: [-8.46169438e-05  4.67591200e-01 -3.24182276e-02  1.23484928e-01
 -1.51356361e-01  8.61481425e-01]
tau[:, i]: [893.5082 907.7194 928.7673 901.6058 866.2244]
B[i]: 236.1327
tau_perturbed[:, i]: [893.5082 907.8965 928.755  901.6526 866.1671]
B_perturbed[i]: 236.4589
tau_diffs: [0.00917497 0.07099655 0.03663086 0.00704699]
B_diffs: [0.0009 0.0207 0.0018 0.0009]

Iter: 985
w_i: [-0.54376548 -0.45287039 -0.09265245 -0.50528626 -0.29567618  0.38458345]
tau[:, i]: [893.5082 907.583  928.7768 901.5698 866.2685]
B[i]: 235.8815
tau_perturbed[:, i]: [893.3024 907.4116 928.7417 901.3785 866.1566]
B_perturbed[i]: 236.0271
tau_diffs: [0.07099655 0.03663086 0.00704699 0.16981331]
B_diffs: [0.0207 0.0018 0.0009 0.1426]

Iter: 986
w_i: [-0.63755517  0.20906155 -0.62354181 -0.21202459 -0.23687087 -0.24484698]
tau[:, i]: [893.5959 907.656  928.7917 901.6513 866.3162]
B[i]: 235.8195
tau_perturbe

In [26]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

first(welfare_list): 1241.3822945733243
min(welfare_list): 537.305998709307
max(welfare_list): 3333.0055027031794
argmin_tau: [6. 6. 0. 6. 6.]
argmin_B: 0.0


In [27]:
y_el_values

{(0, 0, 0): 0.0,
 (0, 0, 1): -9.502683639972531e-09,
 (0, 0, 2): 0.493457714485248,
 (0, 1, 0): 0.0,
 (0, 1, 1): 1.7970120319180709e-09,
 (0, 1, 2): 0.5065422932204237,
 (0, 2, 0): 0.0,
 (0, 2, 1): -8.84933621627251e-10,
 (0, 2, 2): 9.300088893194098e-09,
 (0, 3, 0): 0.0,
 (0, 3, 1): -7.739531210116152e-09,
 (0, 3, 2): 0.49345770430694036,
 (0, 4, 0): 0.0,
 (0, 4, 1): 1.1932470206122187e-08,
 (0, 4, 2): 0.5065422915001208,
 (1, 0, 0): 0.0,
 (1, 0, 1): 3.6040274098425584e-09,
 (1, 0, 2): 0.49734517393691074,
 (1, 1, 0): 0.0,
 (1, 1, 1): 7.865103515537742e-09,
 (1, 1, 2): 0.5026548145939583,
 (1, 2, 0): 0.0,
 (1, 2, 1): -3.738910440222777e-10,
 (1, 2, 2): 9.301618939006658e-09,
 (1, 3, 0): 0.0,
 (1, 3, 1): 8.864202594160986e-09,
 (1, 3, 2): 0.4973451597490076,
 (1, 4, 0): 0.0,
 (1, 4, 1): 5.390190072063206e-09,
 (1, 4, 2): 0.5026548259965996,
 (2, 0, 0): 0.0,
 (2, 0, 1): 6.392217200001937e-09,
 (2, 0, 2): 0.5016743680209568,
 (2, 1, 0): 0.0,
 (2, 1, 1): 2.0003635488534995e-08,
 (2, 1, 2)

In [13]:
y_in_values

{(0, 0, 0): 0.5821719216319861,
 (0, 0, 1): 0.2094961277106064,
 (0, 1, 0): 0.004180742490241613,
 (0, 1, 1): 0.20415120816716587,
 (0, 2, 0): 4.144949098608525e-09,
 (0, 2, 1): 1.1425231528975375e-07,
 (0, 3, 0): 0.5846215247840308,
 (0, 3, 1): 0.20704640616129735,
 (0, 4, 0): 5.016818349357077e-07,
 (0, 4, 1): 0.20833156737283695,
 (1, 0, 0): 0.7108227707122439,
 (1, 0, 1): 4.161077085583072e-09,
 (1, 1, 0): 0.28917722088739817,
 (1, 1, 1): 4.239280934608319e-09,
 (1, 2, 0): 5.664237206604805e-09,
 (1, 2, 1): 1.505274957584457e-07,
 (1, 3, 0): 0.7108226144311154,
 (1, 3, 1): 4.250472527391911e-09,
 (1, 4, 0): 0.28917737714979724,
 (1, 4, 1): 4.168614768110095e-09,
 (2, 0, 0): 1.5326673968831134e-06,
 (2, 0, 1): 1.8587508398815776e-09,
 (2, 1, 0): 0.9999984635749538,
 (2, 1, 1): 1.8988984661208134e-09,
 (2, 2, 0): 7.3891652192127915e-09,
 (2, 2, 1): 2.2095957174396205e-07,
 (2, 3, 0): 1.3042789283025461e-06,
 (2, 3, 1): 1.89848245727403e-09,
 (2, 4, 0): 0.9999986919690309,
 (2, 4, 1):

# Braess Paradox

In [34]:
# TODO: Modify code so that each edge's latency cost is different (can stay the same across lanes.)

# 1. Allow flexible choice of latency functions.
# 2. Increase tolls on other lanes (especially lanes 1 and 3) so much that the competitiveness of \
# the route 0-2-4 increases greatly when the budget is lowered.



time_1 = time.time()

edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
network_Braess = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

# TODO: Fill in below:

tau_1 = np.array([5.0, 5.0, 5.0, 5.0, 5.0])
# tau_1 = np.array([5.0, 10.0, 5.0, 10.0, 5.0])
B_1 = 9.0

v_I_array = np.array([1.0])
v_E_array = np.array([0.5])
a_0 = np.array([0.0, 1.0, 0.0, 0.0, 0.0])
a_1 = np.array([0.95, 0.05, 0.0, 0.0, 0.0])
a_2 = np.array([0.0, 0.05, 0.0, 0.0, 0.0])
a_3 = a_1
a_4 = a_0

a_ex = np.block([[a_0], [a_1], [a_2], [a_3], [a_4]]).T
a_gp = a_ex
print("a_ex:\n", a_ex)

y_el_values_1, y_in_values_1 = solve_CBCP_direct(network_Braess, tau = tau_1, B = B_1, v_I_array = v_I_array, \
                                             v_E_array = v_E_array, a_ex = a_ex, a_gp = a_gp)

print("y_el_values_1:", y_el_values_1)
print()
print("y_in_values_1:", y_in_values_1)
print()

latency_1 = latency_total(network_Braess, tau_1, v_I_array, v_E_array, y_el_values_1, y_in_values_1, \
                          a_ex = a_ex, a_gp = a_gp)
print("latency_1:", latency_1)


time_2 = time.time()

print("Time:", time_2 - time_1)

a_ex:
 [[0.   0.95 0.   0.95 0.  ]
 [1.   0.05 0.05 0.05 1.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]]
y_el_values_1: {(0, 0, 0): 0.836047, (0, 0, 1): -0.0, (0, 0, 2): 0.095522, (0, 1, 0): 0.0, (0, 1, 1): -0.0, (0, 1, 2): 0.068431, (0, 2, 0): 0.127904, (0, 2, 1): -0.0, (0, 2, 2): 0.738552, (0, 3, 0): 0.0, (0, 3, 1): -0.0, (0, 3, 2): 0.065113, (0, 4, 0): 0.836048, (0, 4, 1): -0.0, (0, 4, 2): 0.098838}

y_in_values_1: {(0, 0, 0): 0.0, (0, 0, 1): 0.80099, (0, 1, 0): -0.0, (0, 1, 1): 0.19901, (0, 2, 0): -0.0, (0, 2, 1): 0.598663, (0, 3, 0): 0.0, (0, 3, 1): 0.202327, (0, 4, 0): -0.0, (0, 4, 1): 0.797673}

latency_1: 6.626139453274099
Time: 0.19315505027770996


In [49]:
print("y_el_values_1:", y_el_values_1)
print()
print("y_in_values_1:", y_in_values_1)
print()
print("latency_1:", latency_1)

y_el_values_1: {(0, 0, 0): 0.527322, (0, 0, 1): -0.0, (0, 0, 2): 0.086553, (0, 1, 0): 0.186339, (0, 1, 1): -0.0, (0, 1, 2): 0.199786, (0, 2, 0): 0.0, (0, 2, 1): -0.0, (0, 2, 2): 0.229141, (0, 3, 0): 0.186339, (0, 3, 1): -0.0, (0, 3, 2): 0.198395, (0, 4, 0): 0.527322, (0, 4, 1): -0.0, (0, 4, 2): 0.087944}

y_in_values_1: {(0, 0, 0): 0.0, (0, 0, 1): 0.551054, (0, 1, 0): -0.0, (0, 1, 1): 0.448946, (0, 2, 0): -0.0, (0, 2, 1): 0.100718, (0, 3, 0): 0.0, (0, 3, 1): 0.450336, (0, 4, 0): -0.0, (0, 4, 1): 0.549664}

latency_1: 5.106773477667531


## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['incoming_edges']) == 0]
    self.depth_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['incoming_edges']).issubset(set(edges_accounted))]
        self.depth_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.depths = list(self.depth_edges_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.edges_dict[edge]['start_node'] for edge in self.depth_edges_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['outgoing_edges']) == 0]
    self.height_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['outgoing_edges']).issubset(set(edges_accounted))]
        self.height_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.heights = list(self.height_edges_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.edges_dict[edge]['end_node'] for edge in self.height_edges_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
